In [88]:
#################################################
#created the 26/04/2018 13:42 by Alexis Blanchet#
#################################################


import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import scipy.stats
import matplotlib
import plotly.offline as offline
import plotly.graph_objs as go
pd.set_option('display.max_columns', 500)

In [89]:
CHAINE = '192'
DATE = '2017-12-01'
JOINDATE = "".join(list(DATE.split('-')))

c = list(CHAINE)
n = 4-len(c)
c = ['0']*n
a = "".join(c)
CHAINE2 = a+CHAINE

In [107]:
X_PTV = pd.read_csv('PTV/IPTV_'+CHAINE2+'_'+DATE+'_TF1.csv',index_col=False).drop(['Unnamed: 0'],axis=1)
X_RTS = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-processed.csv',index_col=False)
y = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-y.csv',index_col=False)

In [109]:
X_PTV.head()

,DATEHEURE,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,@CLEDIF,TEMPSFORT,HD,description programme,@DATEMODIF,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,RATIO,CSA,@CLEEMI,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,TITRE,DATE,HEURE,debut
0,2017-12-01T06:25:00,0,0,0,1,TFou,1,0,167439753,0,1,Magazine jeunesse,2017-11-08T02:12:33,12,0,0,0,0,16:9,1,122835453,0,1,0,0,0,0,0,0,403,0,120,0,TFou,2017-12-01,06:25:00,385
1,2017-12-01T08:25:00,0,0,0,1,Météo,1,0,167439754,0,1,Météo,2017-11-07T12:08:12,16,0,0,0,0,16:9,1,122842665,0,0,0,0,0,0,0,0,951,0,5,0,Météo,2017-12-01,08:25:00,505
2,2017-12-01T08:30:00,0,0,0,1,Téléshopping,0,0,167439755,0,1,Magazine de télé-achat,2017-11-07T12:08:12,12,0,0,0,0,16:9,1,122842666,0,0,0,0,0,0,0,0,425,0,45,0,Téléshopping,2017-12-01,08:30:00,510
3,2017-12-01T09:15:00,0,0,0,1,Météo,1,0,167439756,0,1,Météo,2017-11-07T12:08:12,16,0,0,0,0,16:9,1,122842667,0,0,0,0,0,0,0,0,951,0,5,0,Météo,2017-12-01,09:15:00,555
4,2017-12-01T09:20:00,1,0,0,1,Petits secrets entre voisins,1,0,167439757,0,1,Série réaliste,2017-11-07T12:08:12,3,0,0,0,0,16:9,1,116093509,0,0,0,0,0,0,0,0,1026,0,30,0,Petits secrets entre voisins,2017-12-01,09:20:00,560


In [174]:
X_RTS.head()
X_RTS['minutes'] = X_RTS['minutes']+180

In [175]:
X_RTS.shape

(1437, 20)

In [176]:
tdf = pd.DataFrame()
tdf['debut'] = X_PTV['debut']
tdf['fin'] = tdf['debut']+X_PTV['DUREE']
tdf['titre'] = X_PTV['TITRE']
tdf.head()

,debut,fin,titre
0,385,505,TFou
1,505,510,Météo
2,510,555,Téléshopping
3,555,560,Météo
4,560,590,Petits secrets entre voisins


In [177]:
df = pd.DataFrame()
df['minutes'] = (X_RTS['minutes']+180)
df['isinprogramme'] = 0
df['debut'] = 0
df['fin'] = 0

df.head()
    


,minutes,isinprogramme,debut,fin
0,363,0,0,0
1,364,0,0,0
2,365,0,0,0
3,366,0,0,0
4,367,0,0,0


In [181]:
for index, row in df.iterrows():
    for index2, row2 in tdf.iterrows():
        if(int(row2['debut']) <= int(row['minutes']) <= int(row2['fin'])):
            df.set_value(index, 'isinprogramme', 1)
            df.set_value(index, 'fin', row2['fin'])
            df.set_value(index, 'debut', int(row2['debut']))
            df.ix[index,'TITRE']=row2['titre']
            break
        else:
            pass
df[df['isinprogramme']==0]['titre'] = 'en dehors de programmes'

In [182]:
df2 = df

In [183]:
df2 = pd.merge(df, X_RTS, on='minutes', how='outer')

In [184]:
df2[df2['isinprogramme']==0]

,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label
1173,1536,0.0,0.0,0.0,NaN,26379917.0,26278650.0,26160427.0,26166223.0,5796.0,-112427.0,-213694.0,-118223.0,-219490.0,-101267.0,26246304.25,-80081.25,-56213.5,-7.123133e+04,-109745.0,1.865585e-13,0.5,4.965244e-30,-0.0
1174,1537,0.0,0.0,0.0,NaN,26278650.0,26160427.0,26166223.0,26336298.0,170075.0,175871.0,57648.0,5796.0,-112427.0,-118223.0,26235399.50,100898.50,87935.5,1.921600e+04,-56213.5,1.865585e-13,0.5,1.932407e-30,0.0
1175,1538,0.0,0.0,0.0,NaN,26160427.0,26166223.0,26336298.0,26165792.0,-170506.0,-431.0,5365.0,170075.0,175871.0,5796.0,26207185.00,-41393.00,-215.5,1.788333e+03,87935.5,1.865585e-13,0.5,3.448211e-30,0.0
1176,1539,0.0,0.0,0.0,NaN,26166223.0,26336298.0,26165792.0,25867575.0,-298217.0,-468723.0,-298648.0,-170506.0,-431.0,170075.0,26133972.00,-266397.00,-234361.5,-9.954933e+04,-215.5,1.865585e-13,0.5,8.027042e-29,-0.0
1177,1540,0.0,0.0,0.0,NaN,26336298.0,26165792.0,25867575.0,25690069.0,-177506.0,-475723.0,-646229.0,-298217.0,-468723.0,-170506.0,26014933.50,-324864.50,-237861.5,-2.154097e+05,-234361.5,1.865585e-13,0.5,2.725002e-28,-0.0
1178,1541,0.0,0.0,0.0,NaN,26165792.0,25867575.0,25690069.0,25568345.0,-121724.0,-299230.0,-597447.0,-177506.0,-475723.0,-298217.0,25822945.25,-254600.25,-149615.0,-1.991490e+05,-237861.5,1.865585e-13,0.5,2.191893e-28,-0.0
1179,1542,0.0,0.0,0.0,NaN,25867575.0,25690069.0,25568345.0,25373046.0,-195299.0,-317023.0,-494529.0,-121724.0,-299230.0,-177506.0,25624758.75,-251712.75,-158511.5,-1.648430e+05,-149615.0,1.865585e-13,0.5,8.989157e-29,-0.0
1180,1543,0.0,0.0,0.0,NaN,25690069.0,25568345.0,25373046.0,25217573.0,-155473.0,-350772.0,-472496.0,-195299.0,-317023.0,-121724.0,25462258.25,-244685.25,-175386.0,-1.574987e+05,-158511.5,1.865585e-13,0.5,7.732567e-29,-0.0
1181,1544,0.0,0.0,0.0,NaN,25568345.0,25373046.0,25217573.0,25073388.0,-144185.0,-299658.0,-494957.0,-155473.0,-350772.0,-195299.0,25308088.00,-234700.00,-149829.0,-1.649857e+05,-175386.0,1.865585e-13,0.5,9.213996e-29,-0.0
1182,1545,0.0,0.0,0.0,NaN,25373046.0,25217573.0,25073388.0,23156161.0,-1917227.0,-2061412.0,-2216885.0,-144185.0,-299658.0,-155473.0,24705042.00,-1548881.00,-1030706.0,-7.389617e+05,-149829.0,1.865585e-13,0.5,7.223259e-26,-0.0


In [147]:
print(df.shape)
df.head()

(1437, 24)


,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label
0,183,1,100,390,Programmes de la nuit,6360918.0,6410166.0,6408209.0,6377254.0,-30955.0,-32912.0,16336.0,-1957.0,47291.0,49248.0,6389136.75,-11882.75,-16456.0,5445.333333,23645.5,1.865585e-13,0.5,1.173901e-32,0.0
1,184,1,100,390,Programmes de la nuit,6410166.0,6408209.0,6377254.0,6501536.0,124282.0,93327.0,91370.0,-30955.0,-32912.0,-1957.0,6424291.25,77244.75,46663.5,30456.666667,-16456.0,1.865585e-13,0.5,4.854328e-31,0.0
2,185,1,100,390,Programmes de la nuit,6408209.0,6377254.0,6501536.0,6587747.0,86211.0,210493.0,179538.0,124282.0,93327.0,-30955.0,6468686.50,119060.50,105246.5,59846.000000,46663.5,1.865585e-13,0.5,3.411706e-30,0.0
3,186,1,100,390,Programmes de la nuit,6377254.0,6501536.0,6587747.0,6741640.0,153893.0,240104.0,364386.0,86211.0,210493.0,124282.0,6552044.25,189595.75,120052.0,121462.000000,105246.5,1.865585e-13,0.5,2.670442e-29,0.0
4,187,1,100,390,Programmes de la nuit,6501536.0,6587747.0,6741640.0,6822531.0,80891.0,234784.0,320995.0,153893.0,240104.0,86211.0,6663363.50,159167.50,117392.0,106998.333333,120052.0,1.865585e-13,0.5,1.671209e-29,0.0


In [148]:
df = pd.merge(df, X_PTV, on=['debut','TITRE'], how='left')

In [149]:
print(df.shape)
df[500:510]

(1437, 59)


,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,DATEHEURE,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,@CLEDIF,TEMPSFORT,HD,description programme,@DATEMODIF,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,RATIO,CSA,@CLEEMI,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,DATE,HEURE
500,683,1,660,715,Les feux de l'amour,179838994.0,178556391.0,171155784.0,169477300.0,-1678484.0,-9079091.0,-10361694.0,-3498425.14,-8683210.0,-1282603.00,1.747571e+08,-3498425.14,-3498425.14,-3.453898e+06,-3498425.14,1.865585e-13,0.5,2.176602e-23,-1.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
501,684,1,660,715,Les feux de l'amour,178556391.0,171155784.0,169477300.0,168643933.0,-833367.0,-2511851.0,-9912458.0,-1678484.00,-9079091.0,-3498425.14,1.719584e+08,-3314419.00,-1255925.50,-3.304153e+06,-3498425.14,1.865585e-13,0.5,2.455862e-23,-1.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
502,685,1,660,715,Les feux de l'amour,171155784.0,169477300.0,168643933.0,167837865.0,-806068.0,-1639435.0,-3317919.0,-833367.00,-2511851.0,-1678484.00,1.692787e+08,-1440855.50,-819717.50,-1.105973e+06,-1255925.50,1.865585e-13,0.5,2.019021e-25,-0.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
503,686,1,660,715,Les feux de l'amour,169477300.0,168643933.0,167837865.0,167027438.0,-810427.0,-1616495.0,-2449862.0,-806068.00,-1639435.0,-833367.00,1.682466e+08,-1219196.00,-808247.50,-8.166207e+05,-819717.50,1.865585e-13,0.5,5.447029e-26,-0.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
504,687,1,660,715,Les feux de l'amour,168643933.0,167837865.0,167027438.0,166418623.0,-608815.0,-1419242.0,-2225310.0,-810427.00,-1616495.0,-806068.00,1.674820e+08,-1063341.75,-709621.00,-7.417700e+05,-808247.50,1.865585e-13,0.5,3.778062e-26,-0.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
505,688,1,660,715,Les feux de l'amour,167837865.0,167027438.0,166418623.0,166105577.0,-313046.0,-921861.0,-1732288.0,-608815.00,-1419242.0,-810427.00,1.668474e+08,-741798.75,-460930.50,-5.774293e+05,-709621.00,1.865585e-13,0.5,1.554347e-26,-0.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
506,689,1,660,715,Les feux de l'amour,167027438.0,166418623.0,166105577.0,165588521.0,-517056.0,-830102.0,-1438917.0,-313046.00,-921861.0,-608815.00,1.662850e+08,-696518.75,-415051.00,-4.796390e+05,-460930.50,1.865585e-13,0.5,6.461682e-27,-0.0,2017-12-01T11:00:00,0.0,0.0,0.0,1.0,Les feux de l'amour,1.0,0.0,167439760.0,0.0,1.0,Feuilleton sentimental,2017-11-18T03:16:26,4.0,0.0,0.0,0.0,0.0,16:9,1.0,120971283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,55.0,0.0,2017-12-01,11:00:00
507,690,1,660,715,Les feux de l'amour,166418623.0,166105577.0,165588521.0,165058738.0,-

In [150]:
df = df.drop(['@DATEMODIF','@CLEDIF','DATEHEURE','RATIO'],axis=1)

In [151]:
df[df['isinprogramme']==0]

,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,TEMPSFORT,HD,description programme,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,CSA,@CLEEMI,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,DATE,HEURE
1353,1536,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1354,1537,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1355,1538,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1356,1539,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1357,1540,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358,1541,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1359,1542,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,1543,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1361,1544,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1362,1545,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
